# AI2023_C6_Project: Distribusi Strategis Tangki Air
Ketua: 
- Hugo Alfedo Putra (225150201111013)

Anggota:
- Farrel Rakha Dzakwan (NIM)
- Kartika Madania (NIM)
- Nafakhatul Fadhliyah (NIM)
- Rayhan Egar Sadtya N. (225150201111014)

## Project Description

- Detailed proposal: <a href="https://drive.google.com/file/d/17-KTcC_7lHK3ccReEwo3SYMl73sKyIs8/view?usp=sharing">Click Here (UB Account)</a>
- Case example: <a href="https://drive.google.com/file/d/1cw2pzo-zphH3vdL3rC1ZDHoAic22058L/view?usp=sharing">Click Here (UB Account)</a>

## Dependencies

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import heapq

## Class: Node
Node mereperesentasikan sebuah desa, memiliki atribut seperti:
- (str) name 
- (int[]) coordinate
- (int) population
- (object[]) neighbors

In [2]:
class Node:
    
    # initialization function
    def __init__(self, name, coordinate_x, coordinate_y, population):
        self.name = name
        self.coordinate = np.array([coordinate_x, coordinate_y])
        self.population = population
        self.neighbors = []
    
    # This is to take care when comparing (k_1, Node_1) < (k_2, Node_2) where k_1 = k_2. 
    # Such case where k_1 = k_2 would trigger heapq to compare the Node object. Comparing 
    # such object would raise an error because we did not and we can not put a comparator 
    # in the Node class. Therefore, this function below is to say that if such case happens, 
    # *always* prioritise the more populous node.
    def __lt__(self, other):
        return self.population < other.population

    # Coordinates are more unique than names
    def __hash__(self):
        return hash(self.coordinate[0])

    # Establishes bi-directional path
    def add_neighbor(self, neighbor):
        # The if-condition below guarantees no recursive connections
        # i.e. a node can't be neighbors with itself
        if self.coordinate[0] != neighbor.coordinate[0]:
            sld = np.sqrt(np.power(neighbor.coordinate[0] - self.coordinate[0], 2) + np.power(neighbor.coordinate[1] - self.coordinate[1] ,2))
            cost = random.choice([i for i in range(100)])
            self.neighbors.append([neighbor, sld, cost])
            neighbor.neighbors.append([self, sld, cost])
    
    # Get neighbors
    def get_neighbor(self):
        its_neighbors = []
        for i in range(len(self.neighbors)):
            its_neighbors.append(self.neighbors[i][0].name)
        return its_neighbors
    
    def get_cost(self):
        its_costs = []
        for i in range(len(self.neighbors)):
            its_costs.append(self.neighbors[i][2])
        return its_costs
    
    def get_sld(self):
        its_slds = []
        for i in range(len(self.neighbors)):
            its_slds.append(self.neighbors[i][1])
        return its_slds

    # Print node information
    def print_info(self):
        print(f"Name: {self.name}")
        print(f"Coordinate [X, Y]: {self.coordinate}")
        print(f"Population: {self.population}")
        print(f"Neighbors: {self.get_neighbor()}")
        print(f"Cost: {self.get_cost()}")
        print(f"SLD: {self.get_sld()}")

    # Quick debug purposes
    def minimal_print(self):
        print(f"ID: {self.name} \t -> {self.get_neighbor()}")

In [3]:
node1 = Node("A", 1, 2, 3)
node2 = Node("B", 2, 3, 4)
node1.add_neighbor(node2)
print(node1.neighbors[0][2])

10


## Class: Environment

In [4]:
class Environment:
    
    # Initialization function
    def __init__(self, *args):

        # Random generation needs name, area_size, node_number
        if len(args) > 1 and isinstance(args[0], str):
            self.name = args[0]
            self.node_number = args[2]
            self.empty_node_avail = args[2] # kind of redundant (read more below)
            self.area_size = args[1]
            self.area = [[None for _ in range(args[1])] for _ in range(args[1])]

            self.node_list = [] # Used to store generated nodes
            # Note: making an array filled with None and then *appending* instead of *updating*
            # the values doesn't make any sense because you'd have Nones in front of everything else
            
            self.node_score = [None for _ in range(args[2])] # Used to store f(n) values for generated nodes

        # Nodes generated from data will only need Node
        if len(args) == 1 and isinstance(args[0], list):
            self.nodes = args[0]
            self.node_number = len(args[0])
            self.node_list = [] 
            self.node_score = [None for _ in range(len(args[0]))]
    
    # Add node
    def add_node(self, node):
        self.area[node.coordinate[0]][node.coordinate[1]] = node
        self.node_list.append(node)
        self.empty_node_avail -= 1 # you have a while loop using "remaining" as a decreasing variable, I don't think this is useful
    
    def add_node_from_data(self, node):
        self.node_list.append(node)

    # Generate test case environment
    def test_case(self):
        O_node = Node("O", 0, 0, 0)
        A_node = Node("A", -3, 3, 300)
        B_node = Node("B", 1, 6, 100)
        C_node = Node("C", 5, 5, 150)
        D_node = Node("D", 2, 3, 400)
        E_node = Node("E", 5, 3, 250)
        F_node = Node("F", 4, 0, 75)
        self.add_node(O_node)
        self.add_node(A_node)
        self.add_node(B_node)
        self.add_node(C_node)
        self.add_node(D_node)
        self.add_node(E_node)
        self.add_node(F_node)
        O_node.add_neighbor(D_node)
        O_node.add_neighbor(F_node)
        A_node.add_neighbor(B_node)
        A_node.add_neighbor(D_node)
        B_node.add_neighbor(A_node)
        B_node.add_neighbor(D_node)
        B_node.add_neighbor(C_node)
        C_node.add_neighbor(B_node)
        C_node.add_neighbor(E_node)
        D_node.add_neighbor(E_node)
        D_node.add_neighbor(F_node)
        self.node_number = 7

    # Populate area 
    def populate_area(self):
        # Makes sure the origin node is registered in node_list and area after creation
        new_node = Node("origin", 0, 0, 0)
        self.add_node(new_node)

        remaining = self.node_number
        naming_counter = 0

        # Generate randomly-placed nodes
        while(remaining > 0):

            # Randomize index
            x_loc = np.random.randint(1, self.area_size)
            y_loc = np.random.randint(1, self.area_size)

            # If index empty, initialize node
            if (self.area[x_loc][y_loc] is None):
                # Creates data for new_node
                node_name = str(self.name) + "node" + str(naming_counter) 
                naming_counter += 1
                node_population = np.random.randint(50, 1000)

                # Creates the new_node
                new_node = Node(node_name, x_loc, y_loc, node_population)

                # Adds the node using a function *you* have created before
                self.add_node(new_node)

                remaining -= 1

        # Generate randomly-put edges (neighbors) between nodes
        # Constraint: 
        #   1. Guaranteed edge from "origin" to at least one node
        #   2. Every node is reachable

        # choice is a list containing all possible integers from 0 to self.node_number
        choice = [i for i in range(self.node_number)]

        for i in range(len(self.node_list)-1):
            # Makes sure there is a path from origin to the last node where
            # it visits every other nodes on the way (satisfies all constraints)
            self.node_list[i].add_neighbor(self.node_list[i+1])

            # Connects one node with random neighbors if it passes a certain
            # threshold (chance > k)
            chance = np.random.random()
            if chance > 0.69: # change this if you want
                self.node_list[i].add_neighbor(self.node_list[random.choice(choice)])

    def populate_from_data(self):
        # Pick a place with the heighest population count as the origin
        max_pop = 0; idx = 0
        for i in range(len(self.nodes)):
            # Finds the place with the highest population count and gets its index
            if self.nodes[i].population > max_pop:
                max_pop = self.nodes[i].population
                idx = i
        # Commits it
        self.add_node_from_data(self.nodes[idx])

        # Add the rest of the nodes
        for i in range(len(self.nodes)):
            if i != idx:
                self.add_node_from_data(self.nodes[i])
            else: continue

        # Add random neighbors to each other 
        # Generate randomly-put edges (neighbors) between nodes
        # Constraint: 
        #   1. Guaranteed edge from "origin" to at least one node
        #   2. Every node is reachable

        # choice is a list containing all possible integers from 0 to self.node_number
        choice = [i for i in range(len(self.nodes))]

        for i in range(len(self.nodes)-1):
            if i != idx:
                # Makes sure there is a path from origin to the last node where
                # it visits every other nodes on the way (satisfies all constraints)
                self.nodes[i].add_neighbor(self.nodes[i+1])

                # Connects one node with random neighbors if it passes a certain
                # threshold (chance > k)
                chance = np.random.random()
                if chance > 0.69: # change this if you want
                    self.nodes[i].add_neighbor(self.nodes[random.choice(choice)])

### Environment Use Case

In [5]:
env = Environment("test", 16, 16)
env.test_case()
for i in range(env.node_number):
    env.node_list[i].minimal_print()

ID: O 	 -> ['D', 'F']
ID: A 	 -> ['B', 'D', 'B']
ID: B 	 -> ['A', 'A', 'D', 'C', 'C']
ID: C 	 -> ['B', 'B']
ID: D 	 -> ['O', 'A', 'B', 'E', 'F']
ID: E 	 -> ['D']
ID: F 	 -> ['O', 'D']


### Referensi Geolokasi Database

Teks di bawah ini sekadar mencontohkan cara kerja kelompok kami:

Didapatkan sebuah batasan berupa koordinat-koordinat dari website https://boundingbox.klokantech.com sebagai berikut:
- batasan Barat: `112.471417`
- batasan Selatan: `-8.048836`
- batasan Timur: `112.745595`
- batasan Utara: `-7.80712`

dengan visualisasi:
<br>
<img src="./visualisasi_1.png" width="800">

dan juga database `ref_db.csv` yang akan digunakan untuk menentukan desa-desa mana saja yang berada di dalam batasan-batasan tersebut.


## Tahapan Kerja

1. Deklarasi batasan

In [6]:
batasan = [112.599811,-7.978172,112.668399,-7.92068]
# dengan urutan barat, selatan, timur, utara

2. Mengambil data dari .csv untuk melakukan referensi geolokasi

In [7]:
ref_db = pd.read_csv('ref_db.csv')
ref_db.head()

,name,lat,lng,pop
0,LATIUNG,2.357417,96.452420,779
1,LABUHAN BAJAU,2.391348,96.479117,88
2,SUAK LAMATAN,2.353943,96.364809,2205
3,ANA AO,2.439100,96.450882,1890
4,LATALING,2.458705,96.455400,1577


3. Membuat fungsi untuk mencari batasan desa

In [8]:
def cari_desa(batasan):
    kumpulan_desa = []
    for i in range(len(ref_db)):
        if ref_db['lat'][i] >= batasan[1] and ref_db['lat'][i] <= batasan[3] and ref_db['lng'][i] >= batasan[0] and ref_db['lng'][i] <= batasan[2] and ref_db['lat'][i] != 0 and ref_db['lng'][i] != 0:
            new_node = Node(ref_db['name'][i],ref_db['lat'][i],ref_db['lng'][i],ref_db['pop'][i])
            kumpulan_desa.append(new_node)
        else: continue
    return kumpulan_desa

4. Melakukan generasi environment

In [9]:
env_on_data = Environment(cari_desa(batasan))
env_on_data.populate_from_data()
# NEVER print on cells where you declare or instantiate a class; it will double up

In [10]:
print(env_on_data.node_number)
print("---")
for i in range(env_on_data.node_number):
    env_on_data.node_list[i].minimal_print()
print("---")
env_on_data.node_list[0].minimal_print()

27
---
ID: MANGLIAWAN 	 -> ['DADAPAN']
ID: DADAPAN 	 -> ['MOJOASEM', 'MANGLIAWAN']
ID: MOJOASEM 	 -> ['DADAPAN', 'MRIYUNAN']
ID: MRIYUNAN 	 -> ['MOJOASEM', 'WADENG']
ID: WADENG 	 -> ['MRIYUNAN', 'JUNGANYAR']
ID: JUNGANYAR 	 -> ['WADENG', 'SOCAH', 'PETAONAN']
ID: SOCAH 	 -> ['JUNGANYAR', 'JADDIH']
ID: JADDIH 	 -> ['SOCAH', 'KELEYAN']
ID: KELEYAN 	 -> ['JADDIH', 'PETAONAN']
ID: PETAONAN 	 -> ['KELEYAN', 'DAKIRING', 'JUNGANYAR']
ID: DAKIRING 	 -> ['PETAONAN', 'PERNAJUH']
ID: PERNAJUH 	 -> ['DAKIRING', 'TOBAI BARAT']
ID: TOBAI BARAT 	 -> ['PERNAJUH', 'TOBAI TENGAH']
ID: TOBAI TENGAH 	 -> ['TOBAI BARAT', 'BIRA TENGAH', 'BIRA TENGAH']
ID: BIRA TENGAH 	 -> ['TOBAI TENGAH', 'BIRA TIMUR', 'TOBAI TENGAH']
ID: BIRA TIMUR 	 -> ['BIRA TENGAH', 'SOKOBANAH LAOK']
ID: SOKOBANAH LAOK 	 -> ['BIRA TIMUR', 'TAMBERU LAOK']
ID: TAMBERU LAOK 	 -> ['SOKOBANAH LAOK', 'TAMBERU DAYA']
ID: TAMBERU DAYA 	 -> ['TAMBERU LAOK', 'SOKOBANAH TENGAH']
ID: SOKOBANAH TENGAH 	 -> ['TAMBERU DAYA', 'SOKOBANAH DAYA']
ID: SOKOB

## Environment Evaluation

A* Search, expand minimum $f(n) = g(n) + h(n)$ node first.

$$ g(n) = cost(root, n) + \frac{\sum _{i = 0}^n cost(i, n)}{n-1} $$

$$ h(n) = \frac{\sum _{i=0}^n h_{SLD}(i)}{n-1} + \frac{\sum _{i = 0}^{n-1}\text{population}(h) * d_{SLD}(i, k)}{\sum _{i=0}^n \text{population}(h)} $$

$$ f(n) = cost(root, n) + \frac{\sum _{i = 0}^n cost(i, n)}{n-1} + \frac{\sum _{i=0}^n h_{SLD}(i)}{n-1} + \frac{\sum _{i = 0}^{n-1}\text{population}(h) * d_{SLD}(i, k)}{\sum _{i=0}^n \text{population}(h)} $$

### Fungsi $g(n)$

In [11]:
def find_index(array, target_node):
    try:
        index = array.index(target_node)
        return index
    except ValueError:
        return None

def find_cost(graph, start_node):
    distances = {node: float('infinity') for node in graph}
    distances[start_node] = 0
    previous_nodes = {node: None for node in graph}

    priority_queue = [(0, start_node)]

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_distance > distances[current_node]:
            continue

        for properties in graph[find_index(graph, current_node)].neighbors:
            # neighbor = properties[0]
            # weight = properties[2]
            distance = current_distance + properties[2]
            if distance < distances[properties[0]]:
                distances[properties[0]] = distance
                previous_nodes[properties[0]] = current_node
                heapq.heappush(priority_queue, (distance, properties[0]))

    return [distances, previous_nodes]

def gn(environment):
    iter = environment.node_number
    graph = environment.node_list
    origin = graph[0] # Node at index 0 is always the origin node
    gn_values = []

    # Discovers the graph from the origin
    o_dist, o_prev = find_cost(graph, origin)

    min = float('inf'); idx = 0
    for i in range(1, iter): # We don't need to do calculations on the origin node
        # Define current node
        current_node = graph[i]

        # Find the first term
        term_1 = o_dist[current_node]

        # Find the second term
        term_2 = 0
        dist, prev = find_cost(graph, current_node)
        for j in range(1, iter):
            if i != j: term_2 += dist[graph[j]]
            else: continue 
        term_2 /= (iter - 1)
        
        # Add them together
        gn_term = term_1 + term_2
        gn_values.append(gn_term)

        # Find minimum
        if gn_term < min:
            min = gn_term
            idx = i

    return gn_values

### Fungsi $f(n)$

In [12]:
def hn(environment):
    iter = environment.node_number
    graph = environment.node_list
    origin = graph[0] # Node at index 0 is always the origin node
    hn_values = []

    # Find the total population
    pop_sum = 0
    for i in range(1, iter):
        pop_sum += graph[i].population
    
    min = float('inf'); idx = 0
    for i in range(1, iter):
        term_1 = 0
        term_2 = 0
        for j in range(iter - 1):
            if i != j: 
                sld = np.sqrt((graph[j].coordinate[0] - graph[i].coordinate[0]) ** 2 + (graph[j].coordinate[1] - graph[i].coordinate[1]) ** 2)
                # Calculating first term
                term_1 += sld
                # Calculating second term because second term needs the result from the first term
                term_2 += graph[j].population * sld
            else: continue
        term_1 /= (iter - 1)
        term_2 /= pop_sum
        
        hn_term = term_1 + term_2
        hn_values.append(hn_term)

        # Find minimum
        if hn_term < min:
            min = hn_term
            idx = i
    
    return hn_values

### Mencari $f(n)$ minimum

In [13]:
def fn(environment):
    fn_values = []
    gn_values = gn(environment)
    hn_values = hn(environment)

    min = float('inf'); idx = 0
    for i in range(len(gn_values)):
        fn = gn_values[i] + hn_values[i]
        fn_values.append(fn)
        if fn < min:
            min = fn
            idx = i

    return [min, idx], fn_values

### Eksekusi

In [14]:
min, vals = fn(env_on_data)
print(f'Penempatan truk tangki air di {env_on_data.node_list[min[1]].name} paling efisien dengan nilai f(n) = {min[0]}')

Penempatan truk tangki air di MANGLIAWAN paling efisien dengan nilai f(n) = 530.3639917844989
